This is the Project Proposal for the DSCI 100 Group Project. 
Group members: Artemis, Cathy, Abdulaziz Umiryayev

The title of the project is: Patient Survival Predictor
-> The dataset is drawn from information received from in-hospital mortality for admitted patients.
The model would answer the following predictive question: Based on the foundational characteristics, what are the chances that the admitted patient dies?

-> foundational characteristics would be figured out through trial-and-error method of finding which characteristics have the strongest correlation with the death occurence.

In [5]:
https://www.kaggle.com/mitishaagarwal/patient
https://canvas.ubc.ca/courses/83542/assignments/1106463


ERROR: Error in parse(text = x, srcfile = src): <text>:1:7: unexpected '/'
1: https:/
          ^


In [6]:
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source("cleanup.R")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [7]:
dataset <- read_csv('data/WineQT.csv')
#renames the columns and gets rid of the spaces
colnames(dataset)<- c('fix_acid','vol_acid','citric_acid','resid_sugar','chlorides','free_sugar_dioxide','total_sulfur_dioxide','density','pH','sulphates','alcohol','quality','id')
upd <- dataset %>%
         #turns quality variable into a factor
         mutate(quality = as_factor(quality))

#moves ID column at the beginning 
updated_dataset <- upd[,c(13,1,2,3,4,5,6,7,8,9,10,11,12)]
updated_dataset


Parsed with column specification:
cols(
  `fixed acidity` = col_double(),
  `volatile acidity` = col_double(),
  `citric acid` = col_double(),
  `residual sugar` = col_double(),
  chlorides = col_double(),
  `free sulfur dioxide` = col_double(),
  `total sulfur dioxide` = col_double(),
  density = col_double(),
  pH = col_double(),
  sulphates = col_double(),
  alcohol = col_double(),
  quality = col_double(),
  Id = col_double()
)



id,fix_acid,vol_acid,citric_acid,resid_sugar,chlorides,free_sugar_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
0,7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,5
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
1594,6.2,0.600,0.08,2.0,0.090,32,44,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39,51,0.99512,3.52,0.76,11.2,6
1597,5.9,0.645,0.12,2.0,0.075,32,44,0.99547,3.57,0.71,10.2,5


In [8]:
wine_split <- initial_split(updated_dataset, prop = 0.75, strata = quality)
wine_train <- training(wine_split)
wine_test <- testing(wine_split)

wine_train

id,fix_acid,vol_acid,citric_acid,resid_sugar,chlorides,free_sugar_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
0,7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,5
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
1594,6.2,0.600,0.08,2.0,0.090,32,44,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39,51,0.99512,3.52,0.76,11.2,6
1597,5.9,0.645,0.12,2.0,0.075,32,44,0.99547,3.57,0.71,10.2,5


In [68]:

            
wine_summarize <- wine_train%>% 
                       group_by(quality)%>%
                       summarise(vol_acid_avg = median(vol_acid), alcohol_avg = median(alcohol), total_sulfur_dioxide_avg = median(total_sulfur_dioxide),sulphates_avg = median(sulphates), density_avg = median(density), count = n())


                        
#the quality distribution falls into the Gaussian normal distribution
#
wine_summarize

`summarise()` ungrouping output (override with `.groups` argument)



quality,vol_acid_avg,alcohol_avg,total_sulfur_dioxide_avg,sulphates_avg,density_avg,count
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
3,0.8175,9.825,15,0.56,0.997565,6
4,0.6750,9.900,32,0.57,0.996700,25
5,0.5900,9.700,46,0.58,0.996890,363
6,0.5000,10.600,35,0.63,0.996500,349
7,0.3800,11.700,28,0.75,0.995860,105
8,0.3750,12.550,23,0.73,0.994890,10
